In [7]:
import cv2
import mediapipe as mp
import numpy as np
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle

In [16]:
mp_drawing = mp.solutions.drawing_utils 
mp_holistic = mp.solutions.holistic

In [9]:
%config Completer.use_jedi = False

In [10]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [14]:
def findCords(value, coordinates):
    if(value=='bicep_curls'):
        shoulder=[coordinates[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].x, coordinates[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].y]
        elbow=[coordinates[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].x, coordinates[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].y] 
        wrist=[coordinates[mp.solutions.pose.PoseLandmark.LEFT_WRIST.value].x, coordinates[mp.solutions.pose.PoseLandmark.LEFT_WRIST.value].y]
        angle=calculateAngle(shoulder, elbow, wrist)
        return angle
    elif(value):
        return int(0)

In [12]:
# load pkl file
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [13]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [19]:
cap = cv2.VideoCapture("BC1.mp4")

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Curl counter variables
counter = 0 
stage = None
   
size = (frame_width, frame_height)

# output = cv2.VideoWriter('bicep_curls_pred.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, size)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        


        #  Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            row = pose_row

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            
            calcAngle = findCords(body_language_class, results.pose_landmarks.landmark)
            
            # Curl counter logic
            if calcAngle > 140:
                stage = "down"
            if calcAngle < 50 and stage =='down':
                stage="up"
                counter +=1
                print(counter)
            
            cv2.rectangle(image, (0,60), (290, 120), (0, 117, 0), -1)
            
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(image, "COUNTER : ", (15,100), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(counter), (200,110), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2, cv2.LINE_AA)
#             cv2.putText(image, str(calcAngle),
#                        tuple(np.multiply(elbow,[640,480]).astype(int)),
#                             cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA
#                        )
#             print(coordinates)
            
            
            # Get status box
            cv2.rectangle(image, (0,0), (290, 60), (245, 117, 16), -1)
            
            
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            break
                        
        cv2.imshow('Raw Webcam Feed', image)
#         output.write(image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
# output.release()
# print("The video was successfully saved")
cv2.destroyAllWindows()